In [ ]:
# Initializing the project and the keys
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "5f00e1ce-d082-4a2b-b08f-889658e932b7"

fablib = fablib_manager(project_id=project_id)

# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
#fablib.show_config()
import json
import traceback

fablib.verify_and_configure()
!chmod 0600 fabric_config/fabric_bastion_key
!chmod 0600 fabric_config/slice_key

slice_name = 'My_P4Kube_slice' 
try:
    # Reinitializes the existing slice
    slice = fablib.get_slice(slice_name)
    slice.list_nodes()
    print('Done retrieving the existing slice')
except:
    print('Slice not found')

In [ ]:
print('INSTALL STRESS_NG on nodes, and download the experiment scripts')
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = f"""
        sudo apt-get install stress-ng -y;
        wget https://raw.githubusercontent.com/gareging/P4Kube/refs/heads/main/cpu_load_test_1_node.sh;
        wget https://github.com/gareging/P4Kube/blob/main/cpu_load_test_2_nodes.sh;
        chmod +x cpu_load_test_1_node.sh;
        chmod +x cpu_load_test_2_nodes.sh;
    """
    node.execute(script)
print('All done, move to the next step')

In [ ]:
print('Install ab benchmarking on the client machine')
node = slice.get_node('client')
node.execute('sudo apt install apache2-utils -y')
print('All done, move to the next step')

In [ ]:
# Script parameters

# Run the stress test
stress = False
# Set the script name to either 'cpu_load_test_1_node.sh' or 'cpu_load_test_2_nodes.sh'
script_name = 'cpu_load_test_1_node.sh'
# Total number of request
total = 1000
# Total number of request
concur = 500
# Timeout (default is 30)
timeout = 30
# URL of the deployment
url = 'http://10.0.0.2/'
# Version of P4Kube you would like to test: LA (Load aware) or NO_LA (Load un-aware)
type_ = "LA"


In [ ]:
print('Setting up plugin based on the selected choice of load-balancing algorithm')
node = slice.get_node('master')

load_aware_string = "LOAD_AWARE" if type_ == "LA" else "NO_LOAD_AWARE"

print('Removing the current plugin instance')
script = "pkill -f kubectl-p4_loadbalancer"
node.execute(script)

print('Restarting the plugin')
script = f'''
echo "Start"
export PATH="/home/ubuntu/P4Kube/kubectl-plugin:/usr/local/go/bin:$PATH"
echo "Path installed"
cd /home/ubuntu/P4Kube/kubectl-plugin || (echo "cd failed" && exit 1)
echo "Directory switched"
echo "Stopped previous plugin run"
cp main.go_{load_aware_string} main.go || (echo "cp failed" && exit 1)
go build -o kubectl-p4_loadbalancer main.go || (echo "go build failed" && exit 1)
nohup ./kubectl-p4_loadbalancer > plugin.log 2>&1 &
echo "Plugin started"
'''
node.execute(script)
print('Enabled', type_)
print("Move to next step")


In [ ]:
print('Starting the load pattern on worker nodes and starting the client')
node_names = [f"node{i}" for i in range(1, 11)]

# Get all node objects from the slice
nodes = [slice.get_node(name) for name in node_names]

# Run the script in background with nohup (survives session end), save PID
if stress:
    for node in nodes:
        print(f"Starting on {node.get_name()}")
        node.execute(f'nohup bash -c "./{script_name} $HOSTNAME" > cpu.log 2>&1 & echo $! > cpu10.pid')
    
print(f'Starting test for {load_aware_string}')
print('Wait until the client node reports about the end of the experiment')
node = slice.get_node('client')
script = f'''nohup ab -n {total} -c {concur} -s {timeout} -l -r {url} > results_{type_}_{total}_c_{concur}.txt &'''
node.execute(script)
print('Experiment is over, move to the next step to see the results')

In [ ]:
print('Results')
print(type_, 'c=', concur, 'total=', total, 'test type', script_name)
node = slice.get_node('client')
node.execute(f'cat results_{type_}_{total}_c_{concur}.txt')

print('Cleaning up')
if stress:
    node_names = [f"node{i}" for i in range(1, 11)]
    nodes = [slice.get_node(name) for name in node_names]
    for node in nodes:
        print(f"Stopping on {node.get_name()}")
        node.execute('sudo kill $(cat cpu10.pid) || true; sudo rm -f cpu10.pid || true;')
        node.execute(f"pkill -f '/bin/bash ./{script_name}' || true")
        node.execute("sudo pkill stress-ng-cpu || true")
        #node.execute('''ps -eo pid,cmd | grep "[c]pu_10.sh" | awk '{print $1}' | xargs kill -9''')
    print('CPU overload stopped')
        
print('Done')